--- Day 14: Disk Defragmentation ---

Suddenly, a scheduled job activates the system's disk defragmenter. Were the situation different, you might sit and watch it for a while, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.

The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were `flqrgnkx`, then the first row would be given by the bits of the knot hash of `flqrgnkx-0`, the second row from the bits of the knot hash of `flqrgnkx-1`, and so on until the last row, `flqrgnkx-127`.

The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of 4 * 32 = 128 bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: 0 becomes `0000`, 1 becomes `0001`, e becomes `1110`, f becomes `1111`, and so on; a hash that begins with `a0c2017...` in hexadecimal would begin with `10100000110000100000000101110000...` in binary.

Continuing this process, the first 8 rows and columns for key `flqrgnkx` appear as follows, using # to denote used squares, and . to denote free ones:

```
##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V   
```

In this example, 8108 squares are used across the entire 128x128 grid.

Given your actual key string, how many squares are used?

Your puzzle input is `oundnydw`.

Borrowing from day 10...

In [1]:
from functools import reduce

def reduce_list_by_numeric_xor(chunk_of_ints):
    return reduce((lambda x, y: x ^ y), chunk_of_ints)

def find_subslice(circle, start, length):
    repeat_slice = circle + circle # adding lists extends them
    subslice = repeat_slice[start:(start+length)]
    subslice.reverse()
    return subslice

def make_new_circle(circle, start, length):
    subslice = find_subslice(circle, start, length)
    repeat_slice = circle + circle # adding lists extends them
    repeat_slice[start:(start+length)] = subslice
    
    # test in case need to wrap circle
    if ((start + length) > len(circle)):
        overflow = (start + length) % len(circle)
        repeat_slice[0:overflow] = repeat_slice[(len(circle)):(len(circle)+overflow)]
    
    keep_circle = repeat_slice[0:len(circle)]
    return keep_circle

def knot_hash(list_of_lengths, input_list_size=256, skip=0, index=0, num_rounds=64):
    input_list = list(range(input_list_size))
    for rd in range(num_rounds):
        for l in list_of_lengths:
            input_list = make_new_circle(input_list, index, l)
            index = (index + l + skip) % len(input_list)
            skip += 1

    chunks = [input_list[i:i+16] for i in range(0, 256, 16)]
    dense_hash = [reduce_list_by_numeric_xor(x) for x in chunks]
    hex_hash = [hex(x).split('x')[-1] for x in dense_hash]
    for i, j in enumerate(hex_hash):
        if len(j) < 2:
            hex_hash[i] = '0' + j
    return hex_hash

In [2]:
test_input = 'flqrgnkx'
my_input = 'oundnydw'

In [3]:
test_rows = [test_input + '-' + str(x) for x in range(128)]
my_rows = [my_input + '-' + str(x) for x in range(128)]

In [4]:
def asciitize(some_string):
    return [ord(x) for x in some_string] + [17, 31, 73, 47, 23]

In [5]:
def knot_hash_to_binary(some_string):
    as_ascii = asciitize(some_string)
    knotted = knot_hash(as_ascii)
    as_hex = ''.join(knotted)
    as_bin = bin(int(as_hex, 16))[2:]
    # pad start with zeroes
    final = (128 - len(as_bin)) * '0' + as_bin 
    return final

In [6]:
test_grid = [knot_hash_to_binary(x) for x in test_rows]

In [7]:
test_grid_as_int = []
for row in test_grid:
    test_grid_as_int.append([int(x) for x in row])

In [8]:
# [len(x) for x in test_grid_as_int]
sum([sum(x) for x in test_grid_as_int])

8108

```
##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
```

In [9]:
test_vals = [
    [1,1,0,1,0,1,0,0],
    [0,1,0,1,0,1,0,1],
    [0,0,0,0,1,0,1,0],
    [1,0,1,0,1,1,0,1],
    [0,1,1,0,1,0,0,0],
    [1,1,0,0,1,0,0,1],
    [0,1,0,0,0,1,0,0],
    [1,1,0,1,0,1,1,0]
]

In [10]:
for x in range(8):
    print(test_grid_as_int[x][:8] == test_vals[x])

True
True
True
True
True
True
True
True


In [11]:
my_grid = [knot_hash_to_binary(x) for x in my_rows]
my_grid_as_int = []
for row in my_grid:
    my_grid_as_int.append([int(x) for x in row])
sum([sum(x) for x in my_grid_as_int])

8106

That's the right answer! You are one gold star closer to debugging the printer.

--- Part Two ---

Now, all the defragmenter needs to know is the number of regions. A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.

```
##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
```

In the example above, the following nine regions are visible, each marked with a distinct digit:

```
11.2.3..-->
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.-->
|      |   
V      V   
```

Of particular interest is the region marked 8; while it does not appear contiguous in this small view, all of the squares marked 8 are connected when considering the whole 128x128 grid. In total, in this example, 1242 regions are present.

How many regions are present given your key string?

Your puzzle input is still `oundnydw`.

In [12]:
columns = []
for y in range(128):
    columns.append([my_grid[x][0] for x in range(128)])

In [13]:
test_sample = []
for x in range(8):
    print(test_grid[x][0:8])
    test_sample.append(list(test_grid[x][0:8]))

11010100
01010101
00001010
10101101
01101000
11001001
01000100
11010110


In [14]:
test_sample

[['1', '1', '0', '1', '0', '1', '0', '0'],
 ['0', '1', '0', '1', '0', '1', '0', '1'],
 ['0', '0', '0', '0', '1', '0', '1', '0'],
 ['1', '0', '1', '0', '1', '1', '0', '1'],
 ['0', '1', '1', '0', '1', '0', '0', '0'],
 ['1', '1', '0', '0', '1', '0', '0', '1'],
 ['0', '1', '0', '0', '0', '1', '0', '0'],
 ['1', '1', '0', '1', '0', '1', '1', '0']]

In [15]:
regions = []
for item in range(8):
    if item == 0:
        regions.append(True)
    else:
        regions.append(test_sample[0][item] != test_sample[0][item-1] and test_sample[0][item] == '1')
regions

[True, False, False, True, False, True, False, False]

In [16]:
regions = [regions]
regions

[[True, False, False, True, False, True, False, False]]

In [20]:
empty_row = [False for x in range(128)]
new_grid = [empty_row for x in range(128)]
len(new_grid[0]) == len(new_grid)

True

In [18]:
test_regions = 0
for row in range(128):
    for col in range(128):
        if test_grid[row][col] == '0': # ignore empty
            continue
        if row == 0 and col == 0: # only need to do this for very first cell
            test_regions += 1
            continue
        if row == 0 and test_grid[row][col] != test_grid[row][col - 1]: # only need to do this for first row
            test_regions += 1
            continue
        if col > 0 and test_grid[row][col] == test_grid[row][col - 1]: # check left
            continue
        if col < 127 and test_grid[row][col] == test_grid[row][col + 1]: # check right
            continue
        if row > 0 and test_grid[row][col] == test_grid[row - 1][col]: # check up
            continue
        if row < 127 and test_grid[row][col] == test_grid[row + 1][col]: # check down
            continue
        test_regions += 1
test_regions

612

In [36]:
def get_regions(input_grid, row, col, output_grid):
    if (row < 0 or col < 0 or row >= 128 or col >= 128 
        or output_grid[row][col] or input_grid[row][col] == '0'):
        return
    # initialize recursive loop
    output_grid[row][col] = True 
    # check left
    get_regions(input_grid, row-1, col, output_grid)
    # check right
    get_regions(input_grid, row+1, col, output_grid)
    # check up
    get_regions(input_grid, row, col-1, output_grid)
    # check down
    get_regions(input_grid, row, col+1, output_grid)


In [47]:
test_regions = 0
new_grid = [[False]*128 for x in range(128)]
for row in range(128):
    for col in range(128):
        if test_grid[row][col] == '1' and not new_grid[row][col]:
            test_regions += 1
            get_regions(test_grid, row, col, new_grid)
test_regions

1242

In [48]:
regions = 0
new_grid = [[False]*128 for x in range(128)]
for row in range(128):
    for col in range(128):
        if my_grid[row][col] == '1' and not new_grid[row][col]:
            regions += 1
            get_regions(my_grid, row, col, new_grid)
regions

1164

That's the right answer! You are one gold star closer to debugging the printer.

You have completed Day 14!